In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import warnings

warnings.filterwarnings("ignore")

import asyncio
import json
import pickle
import time
from glob import glob
from pathlib import Path
from urllib import request

import pandas as pd
from tqdm.autonotebook import tqdm

from nlp_assemblee.scrapping.api import CPCApi
from nlp_assemblee.scrapping.fetch_data_from_api import interventions_of_group, main

In [13]:
# apidep_2007 = CPCApi(legislature="2007-2012")
apidep_2012 = CPCApi(legislature="2012-2017")
apidep_2017 = CPCApi(legislature="2017-2022")
apidep_2022 = CPCApi()

In [33]:
# urls_2012 = await apidep_2012.get_all_interventions_urls(
#     all_pages=False, sort=0, count=500,
#     page=1, verbose=True, max_interventions=1000,
#     save="../data/interventions_urls_by_deputies.json"
# )

In [15]:
# urls_2017 = await apidep_2017.get_all_interventions_urls(
#     all_pages=False, sort=0, count=500,
#     page=1, verbose=True, max_interventions=1000,
#     save="../data/interventions_urls_by_deputies.json"
# )

  0%|          | 0/659 [00:00<?, ?it/s]

In [40]:
# urls_2022 = await apidep_2022.get_all_interventions_urls(
#     all_pages=False, sort=0, count=500,
#     page=1, verbose=True, max_interventions=1000,
#     save="../data/interventions_urls_by_deputies.json"
# )

In [3]:
with open("../data/2012-2017/interventions_urls_by_deputies.json", "rb") as f:
    urls_2012 = json.load(f)
with open("../data/2017-2022/interventions_urls_by_deputies.json", "rb") as f:
    urls_2017 = json.load(f)
with open("../data/last/interventions_urls_by_deputies.json", "rb") as f:
    urls_2022 = json.load(f)

In [4]:
len(urls_2022), len(urls_2017), len(urls_2012)

(599, 659, 648)

In [14]:
api = apidep_2012
urls = urls_2012

deps = api.deputies
slugs = [api.deputies_df[api.deputies_df["nom"] == dep]["slug"].values[0] for dep in deps]
files = glob(f"../data/{api.legislature_name}/interventions/*.json")
files = [file.split("/")[-1].split(".")[0] for file in files]

idx = [i for i, slug in enumerate(slugs) if slug not in files]

for dep, slug in tqdm(zip(deps, slugs)):
    try:
        await api.async_fetch_interventions_of_deputy(
            dep_urls=urls[dep],
            slug_name=slug,
            max_interventions=1000,
            verbose=False,
            save="../data/",
        )
    except Exception as e:
        print(e)
        time.sleep(60)

  0%|          | 0/648 [00:00<?, ?it/s]

In [18]:
api = apidep_2017
urls = urls_2017

deps = api.deputies
slugs = [api.deputies_df[api.deputies_df["nom"] == dep]["slug"].values[0] for dep in deps]
files = glob(f"../data/{api.legislature_name}/interventions/*.json")
files = [file.split("/")[-1].split(".")[0] for file in files]

idx = [i for i, slug in enumerate(slugs) if slug not in files]
deps = [deps[i] for i in idx]
slugs = [slugs[i] for i in idx]

for dep, slug in tqdm(zip(deps, slugs), total=len(idx)):
    try:
        await api.async_fetch_interventions_of_deputy(
            dep_urls=urls[dep],
            slug_name=slug,
            max_interventions=1000,
            verbose=False,
            save="../data/",
        )
    except Exception as e:
        print(e)
        time.sleep(60)

  0%|          | 0/649 [00:00<?, ?it/s]

[Errno 104] Connection reset by peer
Cannot connect to host 2017-2022.nosdeputes.fr:443 ssl:default [None]


In [22]:
async def fetch_all_interventions(api, urls, save="./data/", max_interventions=1000):
    deps = api.deputies
    slugs = [api.deputies_df[api.deputies_df["nom"] == dep]["slug"].values[0] for dep in deps]
    files = glob(f"{save}/{api.legislature_name}/interventions/*.json")
    files = [file.split("/")[-1].split(".")[0] for file in files]

    idx = [i for i, slug in enumerate(slugs) if slug not in files]
    deps = [deps[i] for i in idx]
    slugs = [slugs[i] for i in idx]

    for dep, slug in tqdm(zip(deps, slugs), total=len(idx)):
        for attempt in range(10):
            try:
                await api.async_fetch_interventions_of_deputy(
                    dep_urls=urls[dep],
                    slug_name=slug,
                    max_interventions=max_interventions,
                    verbose=False,
                    save=save,
                )
            except Exception as e:
                print(e)
                time.sleep(60)
            else:
                break
        else:
            print(f"Failed to fetch {dep} after 10 attempts")
            break

In [23]:
await fetch_all_interventions(apidep_2022, urls_2022, save="../data/")

  0%|          | 0/599 [00:00<?, ?it/s]

[Errno 104] Connection reset by peer
[Errno 104] Connection reset by peer


In [ ]:
with open("../data/last/interventions_urls_by_deputies.json", "rb") as f:
    urls_2022 = json.load(f)

await apidep_2022.async_fetch_all_interventions(urls_2022, save="../data/")